In [ ]:
# Uninstall torchtext
!pip uninstall -y torchtext

# Install Hugging Face datasets
!pip install datasets

# Install compatible versions of torch and torchvision
!pip install torch torchvision

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import random

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Define the Hyperdimensional Computing class
class HDComputing:
    def __init__(self, dim):
        self.dim = dim

    def random_hv(self):
        # Generate a random bipolar hypervector (-1 or 1)
        hv = np.random.choice([-1, 1], size=self.dim)
        return hv

    def superpose(self, hvs):
        # Element-wise addition followed by normalization
        sum_hv = np.sum(hvs, axis=0)
        norm = np.linalg.norm(sum_hv)
        if norm == 0:
            return sum_hv
        return sum_hv / norm

    def bind(self, hv1, hv2):
        # Element-wise multiplication
        return hv1 * hv2

    def permute(self, hv):
        # Circular shift (permutation) to encode position
        return np.roll(hv, 1)

In [ ]:
# Custom Dataset Class for IMDb
class IMDbDataset(Dataset):
    def __init__(self, data, vocab, token_hvs, hd, max_seq_len):
        self.data = data
        self.vocab = vocab
        self.token_hvs = token_hvs
        self.hd = hd
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        label = item['label']
        tokens = text.lower().split()[:self.max_seq_len]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        seq_hv = encode_sequence(tokens, self.token_hvs, self.hd)
        label_tensor = torch.tensor(1.0 if label == 1 else 0.0, dtype=torch.float32)
        return torch.tensor(seq_hv, dtype=torch.float32), label_tensor

# Function to create token hypervectors
def create_token_hvs(vocab, dim, hd):
    token_hvs = {}
    for token in vocab:
        token_hvs[token] = hd.random_hv()
    return token_hvs

# Function to encode sequences into hypervectors
def encode_sequence(tokens, token_hvs, hd):
    position_hv = np.ones(hd.dim)  # Start with a base position hypervector
    sequence_hv = []
    for token in tokens:
        token_hv = token_hvs.get(token, hd.random_hv())  # Handle unknown tokens
        combined_hv = hd.bind(token_hv, position_hv)
        sequence_hv.append(combined_hv)
        position_hv = hd.permute(position_hv)
    # Superpose all token hypervectors to get the sequence representation
    sequence_representation = hd.superpose(sequence_hv)
    return sequence_representation

In [ ]:
# Define the HDC Neural Network model
class HDCNN(nn.Module):
    def __init__(self, dim):
        super(HDCNN, self).__init__()
        self.dim = dim
        # Define a simple feed-forward network
        self.fc1 = nn.Linear(dim, 2048)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(2048, dim)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Forward pass through the network
        out = self.fc1(x)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc2(out)
        # Normalize the output hypervector
        norm = out.norm(p=2, dim=1, keepdim=True)
        norm = torch.where(norm == 0, torch.ones_like(norm), norm)  # Prevent division by zero
        out = out / norm  # Ensure no in-place operation
        return out

In [ ]:
# Main function to train and evaluate the model
def main():
    # Initialize parameters
    dim = 5000  # Reduce dimension to manage memory
    hd = HDComputing(dim)
    max_vocab_size = 5000  # Limit vocabulary size
    max_seq_len = 200  # Limit sequence length
    batch_size = 32
    num_epochs = 5
    learning_rate = 0.001

    # Load IMDb dataset using datasets library
    dataset = load_dataset('imdb')
    train_data = dataset['train']
    test_data = dataset['test']

    # Build vocabulary
    counter = Counter()
    for item in train_data:
        tokens = item['text'].lower().split()
        counter.update(tokens)
    vocab_tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]'] + [token for token, _ in counter.most_common(max_vocab_size)]
    vocab = {token: idx for idx, token in enumerate(vocab_tokens)}

    # Create token hypervectors
    token_hvs = create_token_hvs(vocab, dim, hd)

    # Create datasets
    train_dataset = IMDbDataset(train_data, vocab, token_hvs, hd, max_seq_len)
    test_dataset = IMDbDataset(test_data, vocab, token_hvs, hd, max_seq_len)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss function, and optimizer
    model = HDCNN(dim)
    criterion = nn.CosineEmbeddingLoss()  # Use Cosine Embedding Loss for similarity
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_idx, (inputs, _) in enumerate(train_loader):
            inputs = inputs.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            target = torch.ones(inputs.size(0)).to(device)  # Target is 1 for similar pairs
            loss = criterion(outputs, inputs, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.6f}')

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.6f}')

        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                target = torch.ones(inputs.size(0)).to(device)
                loss = criterion(outputs, inputs, target)
                val_loss += loss.item()
            avg_val_loss = val_loss / len(test_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.6f}')

    # Testing
    model.eval()
    with torch.no_grad():
        similarities = []
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            sim = torch.cosine_similarity(inputs, outputs, dim=1)
            similarities.extend(sim.tolist())

        avg_similarity = sum(similarities) / len(similarities)
        print(f'Average Cosine Similarity on Test Set: {avg_similarity:.4f}')

if __name__ == '__main__':
    main()

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Epoch [1/5], Step [100/782], Loss: 0.217167
Epoch [1/5], Step [200/782], Loss: 0.182535
Epoch [1/5], Step [300/782], Loss: 0.183888
Epoch [1/5], Step [400/782], Loss: 0.155215
Epoch [1/5], Step [500/782], Loss: 0.166116
Epoch [1/5], Step [600/782], Loss: 0.151444
Epoch [1/5], Step [700/782], Loss: 0.171646
Epoch [1/5], Training Loss: 0.190699


KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset, random_split
import random
from collections import Counter

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Define the Hyperdimensional Computing class
class HDComputing:
    def __init__(self, dim):
        self.dim = dim

    def random_hv(self):
        # Generate a random bipolar hypervector (-1 or 1)
        hv = np.random.choice([-1, 1], size=self.dim)
        return hv

    def superpose(self, hvs):
        # Element-wise addition followed by normalization
        sum_hv = np.sum(hvs, axis=0)
        norm = np.linalg.norm(sum_hv)
        if norm == 0:
            return sum_hv
        return sum_hv / norm

    def bind(self, hv1, hv2):
        # Element-wise multiplication
        return hv1 * hv2

    def permute(self, hv):
        # Circular shift (permutation) to encode position
        return np.roll(hv, 1)

# Custom Dataset Class for IMDb
class IMDbDataset(Dataset):
    def __init__(self, data, vocab, token_hvs, hd, max_seq_len):
        self.data = data
        self.vocab = vocab
        self.token_hvs = token_hvs
        self.hd = hd
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        label = item['label']
        tokens = text.lower().split()[:self.max_seq_len]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        seq_hv = encode_sequence(tokens, self.token_hvs, self.hd)
        label_tensor = torch.tensor(1.0 if label == 1 else 0.0, dtype=torch.float32)
        return torch.tensor(seq_hv, dtype=torch.float32), label_tensor

# Function to create token hypervectors
def create_token_hvs(vocab, dim, hd):
    token_hvs = {}
    for token in vocab:
        token_hvs[token] = hd.random_hv()
    return token_hvs

# Function to encode sequences into hypervectors
def encode_sequence(tokens, token_hvs, hd):
    position_hv = np.ones(hd.dim)  # Start with a base position hypervector
    sequence_hv = []
    for token in tokens:
        token_hv = token_hvs.get(token, hd.random_hv())  # Handle unknown tokens
        combined_hv = hd.bind(token_hv, position_hv)
        sequence_hv.append(combined_hv)
        position_hv = hd.permute(position_hv)
    # Superpose all token hypervectors to get the sequence representation
    sequence_representation = hd.superpose(sequence_hv)
    return sequence_representation

# Define the HDC Neural Network model for binary classification
class HDCNNClassifier(nn.Module):
    def __init__(self, dim):
        super(HDCNNClassifier, self).__init__()
        self.dim = dim
        # Define a simple feed-forward network for classification
        self.fc1 = nn.Linear(dim, 2048)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(2048, 1)  # Binary classification output
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Forward pass through the network
        out = self.fc1(x)
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return torch.sigmoid(out)  # Output probability between 0 and 1 for binary classification

# Helper function to build vocabulary
def build_vocab(dataset, max_vocab_size):
    vocab = {}
    counter = Counter()
    for item in dataset:
        tokens = item['text'].lower().split()
        counter.update(tokens)
    vocab_tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]'] + [token for token, _ in counter.most_common(max_vocab_size)]
    return {token: idx for idx, token in enumerate(vocab_tokens)}

# Main function to train and evaluate the model
def main():
    # Initialize parameters
    dim = 5000  # Hypervector dimensionality
    hd = HDComputing(dim)
    max_vocab_size = 5000  # Limit vocabulary size
    max_seq_len = 200  # Limit sequence length
    batch_size = 32
    num_epochs = 5
    learning_rate = 0.001

    # Load IMDb dataset using datasets library
    dataset = load_dataset('imdb')
    train_data = dataset['train']
    test_data = dataset['test']

    # Reduce dataset size to 1/10th
    train_data = train_data.select(range(len(train_data) // 10))
    test_data = test_data.select(range(len(test_data) // 10))

    # Build vocabulary
    vocab = build_vocab(train_data, max_vocab_size)

    # Create token hypervectors
    token_hvs = create_token_hvs(vocab, dim, hd)

    # Create datasets
    train_dataset = IMDbDataset(train_data, vocab, token_hvs, hd, max_seq_len)
    test_dataset = IMDbDataset(test_data, vocab, token_hvs, hd, max_seq_len)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss function, and optimizer
    model = HDCNNClassifier(dim)
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)  # Flatten the output
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.6f}')

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.6f}')

        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            val_loss = 0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs).squeeze(1)
                val_loss += criterion(outputs, labels).item()
                predicted = (outputs >= 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
            avg_val_loss = val_loss / len(test_loader)
            accuracy = correct / total
            print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.6f}, Accuracy: {accuracy:.4f}')

if __name__ == '__main__':
    main()

Epoch [1/5], Training Loss: 0.027183
Epoch [1/5], Validation Loss: 0.000028, Accuracy: 1.0000
Epoch [2/5], Training Loss: 0.000024
Epoch [2/5], Validation Loss: 0.000016, Accuracy: 1.0000
Epoch [3/5], Training Loss: 0.000010
Epoch [3/5], Validation Loss: 0.000007, Accuracy: 1.0000
Epoch [4/5], Training Loss: 0.000002
Epoch [4/5], Validation Loss: 0.000004, Accuracy: 1.0000
Epoch [5/5], Training Loss: 0.000001
Epoch [5/5], Validation Loss: 0.000003, Accuracy: 1.0000
